In [50]:
import numpy as np
import os
import cPickle as pkl
import scipy.signal
import numpy.fft as fft
import sys
import optparse
from PIL import Image
import re
import itertools
from scipy import ndimage

import time
import datetime

import pandas as pd

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure
import csv

import pymworks 
import pandas as pd
import operator
import codecs

import scipy.io
import copy


# Abstract struct class       
class Struct:
    def __init__ (self, *argv, **argd):
        if len(argd):
            # Update by dictionary
            self.__dict__.update (argd)
        else:
            # Update by position
            attrs = filter (lambda x: x[0:2] != "__", dir(self))
            for n in range(len(argv)):
                setattr(self, attrs[n], argv[n])


class cycstruct(Struct):
    times = []
    idxs = 0
    vals = 0
    ordernum = 0
    triggers = 0

def get_timekey(item):
    return item.time


In [51]:


prepend = '/Users/julianarhee'
source_dir = os.path.join(prepend,'nas/volume1/2photon/RESDATA/test_triggers/gratings') #options.source_dir #'/nas/volume1/2photon/RESDATA/TEFO/20160118_AG33/fov1_gratings1'
stimtype = 'grating' #options.stimtype #'grating'
mask = False #options.mask # False
long_trials = False #options.long_trials #True
no_ard = False #options.no_ard

# Look in child dir (of source_dir) to find mw_data paths:
mw_files = os.listdir(os.path.join(source_dir, 'mw_data'))
mw_files = [m for m in mw_files if m.endswith('.mwk')]


# In[6]:


mwfile = mw_files[0]

fn_base = mwfile[:-4]
mw_data_dir = os.path.join(source_dir, 'mw_data')
mw_fn = fn_base+'.mwk'
dfn = os.path.join(mw_data_dir, mw_fn)
dfns = [dfn]

print "MW file: ", dfns



MW file:  ['/Users/julianarhee/nas/volume1/2photon/RESDATA/test_triggers/gratings/mw_data/20161222_JR030W_gratings_10reps_run1.mwk']


In [52]:
def get_session_bounds(dfn):
    df = None
    df = pymworks.open(dfn)                                                          # Open the datafile

    # First, find experiment start, stop, or pause events:
    modes = df.get_events('#state_system_mode')                                      # Find timestamps for run-time start and end (2=run)
    start_ev = [i for i in modes if i['value']==2][0]                                # 2=running, 0 or 1 = stopped

    run_idxs = [i for i,e in enumerate(modes) if e['time']>start_ev['time']]         # Get all "run states" if more than 1 found

    end_ev = next(i for i in modes[run_idxs[0]:] if i['value']==0 or i['value']==1)  # Find the first "stop" event after the first "run" event

    # Create a list of runs using start/stop-event times (so long as "Stop" button was not pressed during acquisition, only 1 chunk of time)
    bounds = []
    bounds.append([start_ev.time, end_ev.time])
    for r in run_idxs[1:]: 
        if modes[r].time < end_ev.time:  # Ignore any extra "run" events if there was no actual "stop" event
            continue
        else:                            # Otherwise, find the next "stop" event if any additional/new "run" events found.
            try:
                stop_ev = next(i for i in modes[r:] if i['value']==0 or i['value']==1)
            except StopIteration:
                end_event_name = 'trial_end'
                print "NO STOP DETECTED IN STATE MODES. Using alternative timestamp: %s." % end_event_name
                stop_ev = df.get_events(end_event_name)[-1]
                print stop_ev
            bounds.append([modes[r]['time'], stop_ev['time']])

    bounds[:] = [x for x in bounds if ((x[1]-x[0])/1E6)>1]
    # print "................................................................"
    print "****************************************************************"
    print "Parsing file\n%s... " % dfn
    print "Found %i start events in session." % len(bounds)
    print "Bounds: ", bounds
    for bidx, bound in enumerate(bounds):
        print "bound ID:", bidx, (bound[1]-bound[0])/1E6, "sec"
    print "****************************************************************"

    return df, bounds

In [53]:
df, bounds = get_session_bounds(dfn)
print bounds

****************************************************************
Parsing file
/Users/julianarhee/nas/volume1/2photon/RESDATA/test_triggers/gratings/mw_data/20161222_JR030W_gratings_10reps_run1.mwk... 
Found 1 start events in session.
Bounds:  [[187058388358, 188268744221]]
bound ID: 0 1210.355863 sec
****************************************************************
[[187058388358, 188268744221]]


In [54]:
def get_trigger_times(df, boundary):
    # deal with inconsistent trigger-naming:
    codec_list = df.get_codec()
    trigger_names = [i for i in codec_list.values() if ('trigger' in i or 'Trigger' in i) and 'flag' not in i]
    if len(trigger_names) > 1:
        print "Found > 1 name for frame-trigger:"
        print "Choose: ", trigger_names
        print "Hint: RSVP could be FrameTrigger, otherwise frame_trigger."
        trigg_var_name = raw_input("Type var name to use: ")
        trigg_evs = df.get_events(trigg_var_name)
    else:
        trigg_evs = df.get_events(trigger_names[0])

    # Only include SI trigger events if they were acquired while MW was actually "running" (i.e., start/stop time boundaries):
    trigg_evs = [t for t in trigg_evs if t.time >= boundary[0] and t.time <= boundary[1]]
    #print trigg_evs

    getout=0
    while getout==0:
        # Find all trigger LOW events after the first onset trigger (frame_trigger=0 when SI frame-trigger is high, =1 otherwise)
        tmp_first_trigger_idx = [i for i,e in enumerate(trigg_evs) if e.value==0][0]        # Find 1st SI frame trigger received by MW (should be "0")
        first_off_ev = next(i for i in trigg_evs[tmp_first_trigger_idx:] if i['value']==1)  # Find the next "frame-off" event from SI (i.e., when MW is waiting for the next DI trigger)
        first_off_idx = [i.time for i in trigg_evs].index(first_off_ev.time)                # Get corresponding timestamp for first SI frame-off event

        # NOTE:  In previous versions of MW protocols, frame-trigger incorrectly reassigned on/off values...
        # Make sure only 1 OFF event for each ON event, and vice versa.
        # Should abort to examine trigger values and tstamps, but likely, will want to take the "frame ON" event immediately before the found "frame OFF" event.
        # (This is because previously, we didn't realize MW's receipt of DI from SI was actually "0" (and frame_trigger was  being used as a flag to turn HIGH, i.e., 1, if trigger received from SI))
        if not trigg_evs[first_off_idx-1].time==trigg_evs[tmp_first_trigger_idx].time:
            print "Incorrect sequence of frame-triggers detected in MW trigger events received from SI:"
            trigg_evs

            # Let USER decide what to do next:
            print "Press <q> to quit and examine. Press <ENTER> to just use frame-ON idx immediately before found frame-OFF idx: "
            user_choice = raw_input()
            valid_response = 0
            while not valid_response:
                if user_choice=='':
                    print "Moving on..."
                    do_quickndirty = True
                    valid_response = 1
                elif user_choice=='q':
                    print "quitting..."
                    do_quickndirty = False
                    valid_response = 1
                else:
                    "Invalid entry provided. Try again."
                    user_choice = raw_input()
            if do_quickndirty:
                first_on_idx = first_off_idx - 1
                first_on_ev = trigg_evs[first_on_idx]
            else:
                print "ABORTING!"
                getout=1
        else:
            first_on_idx = first_off_idx -1
            first_on_ev = trigg_evs[first_on_idx]
            getout=1

    #print "first_on_idx: ", first_on_idx
    print "first on event: ", first_on_ev
    #print "first_off_idx: ", first_off_idx
    print "first off event: ", first_off_ev
    print "Duration of first run: {0:.4f} sec.".format((first_off_ev.time - first_on_ev.time)/1E6)

    # Now, get all "trigger" boundaries that demarcate each "run" after first run:

    found_trigger_evs = [[first_on_ev, first_off_ev]] # placeholder for off ev
    start_idx = copy.copy(first_off_idx)
    print trigg_evs
    while start_idx < len(trigg_evs)-1: 
        print start_idx
        try:
            found_new_start = False
            early_abort = False
            curr_chunk = trigg_evs[start_idx+1:] # Look for next OFF event after last off event 

            # try:
            curr_off_ev = next(i for i in curr_chunk if i['value']==1)
            curr_off_idx = [i.time for i in trigg_evs].index(curr_off_ev.time)
            curr_start_idx = curr_off_idx - 1  # next "frame-start" should be immediately before next found "frame-off" event
            curr_start_ev = trigg_evs[curr_start_idx]
            # if trigg_evs[curr_start_idx]['value']!=0:
            # # i.e., if prev-found ev with value=1 is not a true frame-on trigger (just a repeated event with same value), just ignore it.
            #     continue
            # else:
            found_new_start = True
            # except IndexError:
            #     break

            last_found_idx = [i.time for i in trigg_evs].index(curr_off_ev.time)
            found_trigger_evs.append([curr_start_ev, curr_off_ev])
            start_idx = last_found_idx #start_idx + found_idx
            print start_idx
        except StopIteration:
            print "Got to STOP."
            if found_new_start is True:
                early_abort = True
            break

    # If no proper off-event found for a given start event (remember, we always look for the next OFF event), just use the end of the session as t-end.
    # Since we look for the next OFF event (rather than the next start), if we break out of the loop, we haven't cycled through all the trigg_evs.
    # This likely means that there is another frame-ON event, but not corresponding OFF event.
    if early_abort is True: 
        if found_new_start is True:
            found_trigger_evs.append([curr_chunk[curr_idx], end_ev])
        else:
            found_trigger_evs[-1][1] = end_ev


    trigger_evs = [t for t in found_trigger_evs if (t[1].time - t[0].time) > 1]
    trigger_times = [[t[0].time, t[1].time] for t in trigger_evs]
    print "........................................................................................"
    print "Found %i chunks from frame-on/-off triggers:" % len(trigger_times)
    print "........................................................................................"
    for tidx,trigger in enumerate(trigger_times):
        print tidx, ": ", (trigger[1]-trigger[0])/1E6
    print "........................................................................................"
    runs_selected = 0
    while not runs_selected:
        user_run_selection = raw_input("Select indices [EX: 0,1,2,4] of runs to include, or press <enter> to accept all:\n")
        print user_run_selection
        if any([i>= len(trigger_times) for i in user_run_selection]):
            print "Bad index selected, try again."
            continue
        elif len(user_run_selection)==0:
            user_run_selection = np.arange(0, len(trigger_times))
            runs_selected = 1
        else:
            confirm_selection = raw_input("Press <enter> to accept. Press 'r' to re-try.")
            if confirm_selection=='':
                runs_selected = 1
            else:
                continue
    print "Selected %i runs." % len(user_run_selection)
    trigger_times = [trigger_times[i] for i in user_run_selection]

    return trigger_times


In [55]:
def get_pixelclock_events(df, boundary, trigger_times=trigger_times):
    # Get pixel-clock events:
    tmp_display_evs = df.get_events('#stimDisplayUpdate')                                                  # Get all stimulus-display-update events
    display_evs = [e for e in tmp_display_evs if e.value and not e.value[0]==None]                         # Filter out empty display-update events
    display_evs = [d for d in display_evs if d.time <= boundary[1] and d.time >= boundary[0]]              # Only include display-update events within time boundary of the session

    tmp_pixelclock_evs = [i for i in display_evs for v in i.value if 'bit_code' in v.keys()]                      # Filter out any display-update events without a pixel-clock event
    print "N pix-evs found in boundary: %i" % len(tmp_pixelclock_evs)

    pixelclock_evs = [p for p in tmp_pixelclock_evs if p.time <= trigger_times[-1][1] and p.time >= trigger_times[0][0]] # Make sure pixel events are within trigger times...
    print "Got %i pix code events for current session." % len(pixelclock_evs)
    #pixelevents.append(pixelclock_evs)
    
    return pixelclock_evs


In [58]:
def get_image_events(df, boundary, pixelclock_evs=[], stimtype='grating', mask=False):
    
    # Get all stimulus-udpate events within bounds:
    tmp_display_evs = df.get_events('#stimDisplayUpdate')                                                  # Get all stimulus-display-update events
    display_evs = [e for e in tmp_display_evs if e.value and not e.value[0]==None]                         # Filter out empty display-update events
    display_evs = [d for d in display_evs if d.time <= boundary[1] and d.time >= boundary[0]]              # Only include display-update events within time boundary of the session

    if len(pixelclock_evs)==0:
        pixelclock_evs = [i for i in display_evs for v in i.value if 'bit_code' in v.keys()]
    if len(pixelclock_evs)==0:
        pixelclock_evs = display_evs
       
    # Get stimulus-onset info parsed into trials:
    if stimtype=='image':
        tmp_image_evs = [d for d in pixelclock_evs for i in d.value if 'filename' in i.keys() and '.png' in i['filename']]
        #stimevents.append(imdevs)

        # Find blank ITIs:
        if mask is True:
            iti_evs = [i for i in pixelclock_evs for v in i.value if v['name']=='blue_mask' and i not in tmp_image_evs]
        else:
            iti_evs = [i for i in pixelclock_evs if i.time>image_evs[0].time and i not in tmp_image_evs]

        tmp_trial_evs = tmp_image_evs + iti_evs
        trial_evs = sorted(tmp_trial_evs, key=get_timekey)
        
        image_evs = tmp_image_evs

    elif stimtype=='grating':
        tmp_image_evs = [d for d in display_evs for i in d.value if i['type']=='drifting_grating']

        start_times = [i.value[1]['start_time'] for i in tmp_image_evs] # Use start_time to ignore dynamic pixel-code of drifting grating since stim as actually static
        find_static = np.where(np.diff(start_times) > 0)[0]
        find_static = np.append(find_static, 0)
        find_static = sorted(find_static)
        image_evs = [tmp_image_evs[i+1] for i in find_static]
        #stimevents.append(imtrials)

        # Find blank ITIs:
        if mask is True:
            iti_evs = [i for i in pixelclock_evs if i.time>tmp_image_evs[0].time and i not in tmp_image_evs]
        else:
            prevdev = [[i for i,d in enumerate(display_evs) if d.time < t.time][-1] for t in image_evs[1:]]
            lastdev = [i for i,d in enumerate(display_evs) if d.time > image_evs[-1].time and len(d.value)<3][0] # ignore the last "extra" ev (has diff time-stamp) - just wnt 1st non-grating blank
            iti_evs = [display_evs[i] for i in prevdev]
            iti_evs.append(display_evs[lastdev])
        
        # Check that we got all the blanks:
        blanks = [i for i,p in enumerate(pixelclock_evs) if len(p.value)==2]
        mismatches = [i for i,(p,t) in enumerate(zip([pixelclock_evs[x] for x in blanks], iti_evs)) if not p==t]   
        if len(mismatches)>0:
            print "Mismatches found in parsing trials...."

        # Append a "trial off" at end, if needed:
        if image_evs[-1].time > iti_evs[-1].time: # early-abort
            iti_evs.pop(-1)                
        tmp_trial_evs = image_evs + iti_evs
        trial_evs = sorted(tmp_trial_evs, key=get_timekey)

    #trialevents.append(tmp_trialevents)
    print "Length of trial epochs: ", len(trial_evs)
    print "Number of trials found: ", len(image_evs)
    
    return image_evs, trial_evs

In [60]:
def get_session_info(df, stimtype='grating'):
    info = dict()
    if stimtype=='bar':
        ncycles = df.get_events('ncycles')[-1].value
        info['ncycles'] = ncycles
        info['target_freq'] = df.get_events('cyc_per_sec')[-1].value
        info['barwidth'] = df.get_events('bar_size_deg')[-1].value
    else:
        itis = df.get_events('ITI_time')
        info['ITI'] = itis[-1].value
        sizes = df.get_events('stim_size')
        info['stimsize'] = sizes[-1].value
        # stimulus types?
        # ntrials?
    return info
        

In [63]:
pixelevents = []
stimevents = []
trialevents = []
session_info = dict()
for bidx,boundary in enumerate(bounds):
    bidx = 0
    boundary = bounds[0]
    if (boundary[1] - boundary[0]) < 1000000:
        print "Not a real boundary, only %i seconds found. Skipping." % int(boundary[1] - boundary[0])
        #continue

    print "................................................................"
    print "SECTION %i" % bidx
    print "................................................................"

    trigger_times = get_trigger_times(df, boundary)
    
    pixelclock_evs = get_pixelclock_events(df, boundary, trigger_times=trigger_times)
    
    [image_evs, trial_evs] = get_image_events(df, boundary, pixelclock_evs, stimtype='grating', mask=False)
    
    pixelevents.append(pixelclock_evs)
    trialevents.append(trial_evs)
    stimevents.append(image_evs)
    
    session_info = get_session_info(df)
    

................................................................
SECTION 0
................................................................
Found > 1 name for frame-trigger:
Choose:  ['frame_trigger_timeout', 'frame_trigger_on', 'trigger_received', 'trigger_active', 'frame_trigger', 'frame_triggered']
Hint: RSVP could be FrameTrigger, otherwise frame_trigger.
Type var name to use: frame_trigger
first on event:  Event[code=46, name=frame_trigger, time=187076459353, value=0]
first off event:  Event[code=46, name=frame_trigger, time=187136499413, value=1]
Duration of first run: 60.0401 sec.
[Event[code=46, name=frame_trigger, time=187058388458, value=1], Event[code=46, name=frame_trigger, time=187076459353, value=0], Event[code=46, name=frame_trigger, time=187136499413, value=1], Event[code=46, name=frame_trigger, time=187139675475, value=0], Event[code=46, name=frame_trigger, time=187199755412, value=1], Event[code=46, name=frame_trigger, time=187207515421, value=0], Event[code=46, name=

In [64]:
print session_info

{'stimsize': 50, 'ITI': 2000}


In [67]:
if len(bounds)==1:
    pixelevents = pixelevents[0]
    stimevents = stimevents[0]
    trialevents = trialevents[0]

In [80]:
if stimtype=='grating':
    print "Found %i image trials." % len(image_evs)
    print "Found %i stimulus update events across trials." % len(trial_evs)
    print "Expecting %i TIFFs." % len(trigger_times)

    # GET TRIAL INFO FOR DB:
    trial = dict((i+1, dict()) for i in range(len(image_evs)))
    image_evs = sorted(image_evs, key=get_timekey)
    trial_evs = sorted(trial_evs, key=get_timekey)
    runstart_time = trial_evs[0].time
    for trialidx,iev in enumerate(sorted(image_evs, key=get_timekey)):
        trialnum = trialidx + 1
        blankidx = trialidx*2 + 1
        trial[trialnum]['start_time_ms'] = round(iev.time/1E3)
        trial[trialnum]['end_time_ms'] = round((trial_evs[blankidx].time + session_info['ITI'])/1E3)
        ori = iev.value[1]['rotation']
        sf = iev.value[1]['frequency'] #round(iev.value[1]['frequency'], 2)
        stimname = 'grating-ori-%i-sf-%0.2f' % (ori, sf)
        stimpos = [iev.value[1]['xoffset'], iev.value[1]['yoffset']]
        stimsize = iev.value[1]['height']
        trial[trialnum]['stimuli'] = {'stimulus': stimname, 'position': stimpos, 'scale': stimsize}
        trial[trialnum]['stim_on_times'] = round((iev.time - runstart_time)/1E3)
        trial[trialnum]['stim_off_times'] = round((trial_evs[blankidx].time - runstart_time)/1E3)


Found 350 image trials.
Found 700 stimulus update events across trials.
Expecting 17 TIFFs.


In [81]:
trial[1]

{'end_time_ms': 187077500.0,
 'start_time_ms': 187076497.0,
 'stim_off_times': 1002.0,
 'stim_on_times': 0.0,
 'stimuli': {'position': [0, 0],
  'scale': 50,
  'stimulus': 'grating-ori-75-sf-0.90'}}

0.7